In [1]:
!pip install Basemap

from mpl_toolkits.basemap import Basemap
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy.ma as ma


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.7/860.7 kB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 42.1 MB/s eta 0:00:0000:0100:01
/kaggle/input/quiver-data/25-aug-zonal.txt
/kaggle/input/quiver-data/15-oct-zonal.txt
/kaggle/input/quiver-data/5-aug-zonal.txt
/kaggle/input/quiver-data/15-aug-zonal.txt
/kaggle/input/quiver-data/5-aug-meridonial.txt
/kaggle/input/quiver-data/25-aug-meridonial.txt
/kaggle/input/quiver-data/5-sept-meridonial.txt
/kaggle/input/quiver-data/15-sept-zonal.txt
/kaggle/input/quiver-data/25-sept-meridonial.txt
/kaggle/input/quiver-data/5-sept-zonal.txt
/kaggle/input/quiver-data/25-oct-zonal.txt
/kaggle/input/quiver-data/25-sept-zonal.txt
/kaggle/input/quiver-data/5-oct-meridonial.txt
/kaggle/input/quiver-data/25-oct-meridonial.txt
/kaggle/input/quiver-data/15-oct-meridonial.txt
/kaggle/input/quiver-data/15-sept-meridonial.txt
/kaggle/input/quiver-data/15-aug-meridonial.txt
/kaggle/input/quiver-data/5-oct-zonal.txt

In [2]:
lats_s = []

for i in range (0, 180):
    number = -(89.75 - i*0.50)
    lats_s.append(number)

lats = lats_s
for i in reversed(range(180)):
    lats.append(-lats_s[i])

longs = []
for i in range (0, 721):
    number =  0 + i*0.5
    longs.append(number)

In [3]:
filenames1 = ['/kaggle/input/quiver-data/15-aug-meridonial.txt','/kaggle/input/quiver-data/25-aug-meridonial.txt','/kaggle/input/quiver-data/25-sept-meridonial.txt','/kaggle/input/quiver-data/25-oct-meridonial.txt','/kaggle/input/quiver-data/5-oct-meridonial.txt','/kaggle/input/quiver-data/15-oct-meridonial.txt','/kaggle/input/quiver-data/15-sept-meridonial.txt','/kaggle/input/quiver-data/5-aug-meridonial.txt','/kaggle/input/quiver-data/5-sept-meridonial.txt']
filenames2 = ['/kaggle/input/quiver-data/15-aug-zonal.txt','/kaggle/input/quiver-data/25-aug-zonal.txt','/kaggle/input/quiver-data/25-sept-zonal.txt','/kaggle/input/quiver-data/25-oct-zonal.txt','/kaggle/input/quiver-data/5-oct-zonal.txt','/kaggle/input/quiver-data/15-oct-zonal.txt','/kaggle/input/quiver-data/15-sept-zonal.txt','/kaggle/input/quiver-data/5-aug-zonal.txt','/kaggle/input/quiver-data/5-sept-zonal.txt']
dates = ['Aug 15, 2013', 'Aug 25, 2013', 'Sept 25, 2013', 'Oct 25, 2013', 'Oct 5, 2013', 'Oct 15, 2013', 'Sept 15, 2013', 'Aug 5, 2013', 'Sept 5, 2013']
for k in range (0, 9):
    f1 = open(filenames1[k], 'r')
    f2 = open(filenames2[k], 'r')
    l1 = f1.readlines()
    l2 = f2.readlines()
    f1.close()
    f2.close()
    for i in range (0, 7):
        l1.pop(0)
        l2.pop(0)
    data1 = []
    data2 = []
    for i in range(0, len(l1)):
        vals1 = l1[i].split('\t')
        vals1.pop(0)
        data1.append(vals1)
        vals2 = l2[i].split('\t')
        vals2.pop(0)
        data2.append(vals2)

    data1 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data1]
    data2 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data2]
    valuesz = np.empty(shape=(360, 721))
    meridonial = np.array(data1, dtype=float)
    meridonial[meridonial == -1.0E+34] = 120
    zonal = np.array(data2, dtype = float)
    zonal[zonal == -1.0E+34] = 209
    lon = np.array(longs, dtype = float)
    lat = np.array(lats, dtype = float)
    
    for i in range (0, 360):
        for j in range(0, 721):
            if (meridonial[i][j] != 120 and zonal[i][j] != 209):
                valuesz[i][j]=np.sqrt(meridonial[i][j]**2 + zonal[i][j]**2)
            else:
                valuesz[i][j] = 241
    meridonial = ma.masked_where(meridonial == 120, meridonial)
    zonal = ma.masked_where(zonal == 209, zonal)
    valuesz = ma.masked_where(valuesz == 241, valuesz)
    m = Basemap(projection='cyl', llcrnrlat=-89.875, 
            urcrnrlat=89.75, 
            llcrnrlon=0, 
            urcrnrlon=360,
           resolution='c')
    u_norm = zonal / np.sqrt(meridonial ** 2.0 + zonal ** 2.0)
    v_norm = meridonial / np.sqrt(meridonial ** 2.0 + zonal ** 2.0)
    samples = [5, 10, 15, 20, 25, 40]
    for i in samples:
        cf = m.pcolor(lon,lat,valuesz)
        m.drawcoastlines()
        m.drawparallels(range(-90, 90, 30), labels=[1, 0, 0, 0], fontsize=10, linewidth = 0.25)
        m.drawmeridians(range(-180, 180, 45), labels=[0, 0, 0, 1], fontsize=10, linewidth = 0.25)
        Q = plt.quiver(lon[::i],lat[::i],zonal[::i,::i],meridonial[::i,::i], scale_units='xy', scale=1.5, width=0.0013, color='w')
        plt.title('Wind Speed'+' '+'('+dates[k]+')')
        cb = plt.colorbar(cf, fraction=0.0235, pad=0.03 )
        cb.set_label('m/s',  fontsize=15)
        plt.savefig("{}-quiver-mag-s{}.png".format(dates[k], i), dpi=600)
        plt.clf()

<Figure size 640x480 with 0 Axes>

In [4]:
filenames1 = ['/kaggle/input/quiver-data/15-aug-meridonial.txt','/kaggle/input/quiver-data/25-aug-meridonial.txt','/kaggle/input/quiver-data/25-sept-meridonial.txt','/kaggle/input/quiver-data/25-oct-meridonial.txt','/kaggle/input/quiver-data/5-oct-meridonial.txt','/kaggle/input/quiver-data/15-oct-meridonial.txt','/kaggle/input/quiver-data/15-sept-meridonial.txt','/kaggle/input/quiver-data/5-aug-meridonial.txt','/kaggle/input/quiver-data/5-sept-meridonial.txt']
filenames2 = ['/kaggle/input/quiver-data/15-aug-zonal.txt','/kaggle/input/quiver-data/25-aug-zonal.txt','/kaggle/input/quiver-data/25-sept-zonal.txt','/kaggle/input/quiver-data/25-oct-zonal.txt','/kaggle/input/quiver-data/5-oct-zonal.txt','/kaggle/input/quiver-data/15-oct-zonal.txt','/kaggle/input/quiver-data/15-sept-zonal.txt','/kaggle/input/quiver-data/5-aug-zonal.txt','/kaggle/input/quiver-data/5-sept-zonal.txt']
dates = ['Aug 15, 2013', 'Aug 25, 2013', 'Sept 25, 2013', 'Oct 25, 2013', 'Oct 5, 2013', 'Oct 15, 2013', 'Sept 15, 2013', 'Aug 5, 2013', 'Sept 5, 2013']
for k in range (0, 9):
    f1 = open(filenames1[k], 'r')
    f2 = open(filenames2[k], 'r')
    l1 = f1.readlines()
    l2 = f2.readlines()
    f1.close()
    f2.close()
    for i in range (0, 7):
        l1.pop(0)
        l2.pop(0)
    data1 = []
    data2 = []
    for i in range(0, len(l1)):
        vals1 = l1[i].split('\t')
        vals1.pop(0)
        data1.append(vals1)
        vals2 = l2[i].split('\t')
        vals2.pop(0)
        data2.append(vals2)

    data1 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data1]
    data2 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data2]
    valuesz = np.empty(shape=(360, 721))
    meridonial = np.array(data1, dtype=float)
    meridonial[meridonial == -1.0E+34] = 120
    zonal = np.array(data2, dtype = float)
    zonal[zonal == -1.0E+34] = 209
    lon = np.array(longs, dtype = float)
    lat = np.array(lats, dtype = float)
    
    for i in range (0, 360):
        for j in range(0, 721):
            if (meridonial[i][j] != 120 and zonal[i][j] != 209):
                valuesz[i][j]=np.sqrt(meridonial[i][j]**2 + zonal[i][j]**2)
            else:
                valuesz[i][j] = 241
    meridonial = ma.masked_where(meridonial == 120, meridonial)
    zonal = ma.masked_where(zonal == 209, zonal)
    valuesz = ma.masked_where(valuesz == 241, valuesz)
    m = Basemap(projection='cyl', llcrnrlat=-89.875, 
            urcrnrlat=89.75, 
            llcrnrlon=0, 
            urcrnrlon=360,
           resolution='c')
    u_norm = zonal / np.sqrt(meridonial ** 2.0 + zonal ** 2.0)
    v_norm = meridonial / np.sqrt(meridonial ** 2.0 + zonal ** 2.0)
    samples = [5, 10, 15, 20, 25, 40]
    for i in samples:
        cf = m.pcolor(lon,lat,valuesz)
        m.drawcoastlines()
        m.drawparallels(range(-90, 90, 30), labels=[1, 0, 0, 0], fontsize=10, linewidth = 0.25)
        m.drawmeridians(range(-180, 180, 45), labels=[0, 0, 0, 1], fontsize=10, linewidth = 0.25)
        Q = plt.quiver(lon[::i],lat[::i],u_norm[::i,::i],v_norm[::i,::i], scale_units='xy', scale=0.2, width=0.0013, color='w')
        plt.title('Wind Speed'+' '+'('+dates[k]+')')
        cb = plt.colorbar(cf, fraction=0.0235, pad=0.03 )
        cb.set_label('m/s',  fontsize=15)
        plt.savefig("{}-quiver-norm-s{}.png".format(dates[k], i), dpi=600)
        plt.clf()

<Figure size 640x480 with 0 Axes>

In [5]:
filenames1 = ['/kaggle/input/quiver-data/15-aug-meridonial.txt','/kaggle/input/quiver-data/25-aug-meridonial.txt','/kaggle/input/quiver-data/25-sept-meridonial.txt','/kaggle/input/quiver-data/25-oct-meridonial.txt','/kaggle/input/quiver-data/5-oct-meridonial.txt','/kaggle/input/quiver-data/15-oct-meridonial.txt','/kaggle/input/quiver-data/15-sept-meridonial.txt','/kaggle/input/quiver-data/5-aug-meridonial.txt','/kaggle/input/quiver-data/5-sept-meridonial.txt']
filenames2 = ['/kaggle/input/quiver-data/15-aug-zonal.txt','/kaggle/input/quiver-data/25-aug-zonal.txt','/kaggle/input/quiver-data/25-sept-zonal.txt','/kaggle/input/quiver-data/25-oct-zonal.txt','/kaggle/input/quiver-data/5-oct-zonal.txt','/kaggle/input/quiver-data/15-oct-zonal.txt','/kaggle/input/quiver-data/15-sept-zonal.txt','/kaggle/input/quiver-data/5-aug-zonal.txt','/kaggle/input/quiver-data/5-sept-zonal.txt']
dates = ['Aug 15, 2013', 'Aug 25, 2013', 'Sept 25, 2013', 'Oct 25, 2013', 'Oct 5, 2013', 'Oct 15, 2013', 'Sept 15, 2013', 'Aug 5, 2013', 'Sept 5, 2013']
for k in range (0, 9):
    f1 = open(filenames1[k], 'r')
    f2 = open(filenames2[k], 'r')
    l1 = f1.readlines()
    l2 = f2.readlines()
    f1.close()
    f2.close()
    for i in range (0, 7):
        l1.pop(0)
        l2.pop(0)
    data1 = []
    data2 = []
    for i in range(0, len(l1)):
        vals1 = l1[i].split('\t')
        vals1.pop(0)
        data1.append(vals1)
        vals2 = l2[i].split('\t')
        vals2.pop(0)
        data2.append(vals2)

    data1 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data1]
    data2 = [[s.rstrip('\n') if idx == len(sublist) - 1 else s for idx, s in enumerate(sublist)] for sublist in data2]
    valuesz = np.empty(shape=(360, 721))
    meridonial = np.array(data1, dtype=float)
    meridonial[meridonial == -1.0E+34] = 120
    zonal = np.array(data2, dtype = float)
    zonal[zonal == -1.0E+34] = 209
    lon = np.array(longs, dtype = float)
    lat = np.array(lats, dtype = float)
    
    for i in range (0, 360):
        for j in range(0, 721):
            if (meridonial[i][j] != 120 and zonal[i][j] != 209):
                valuesz[i][j]=np.sqrt(meridonial[i][j]**2 + zonal[i][j]**2)
            else:
                valuesz[i][j] = 241
    meridonial = ma.masked_where(meridonial == 120, meridonial)
    zonal = ma.masked_where(zonal == 209, zonal)
    valuesz = ma.masked_where(valuesz == 241, valuesz)
    m = Basemap(projection='cyl', llcrnrlat=-89.875, 
            urcrnrlat=89.75, 
            llcrnrlon=0, 
            urcrnrlon=360,
           resolution='c')
    d = [3, 6, 9]
    for i in d:
        cf = m.pcolor(lon,lat,valuesz)
        m.drawcoastlines()
        m.drawparallels(range(-90, 90, 30), labels=[1, 0, 0, 0], fontsize=10, linewidth = 0.25)
        m.drawmeridians(range(-180, 180, 45), labels=[0, 0, 0, 1], fontsize=10, linewidth = 0.25)
        Q = plt.streamplot(lon,lat,zonal, meridonial, density = i, color = 'w', linewidth = 0.5, arrowsize=0.3)
        cb = plt.colorbar(cf, fraction=0.0235, pad=0.03 )
        cb.set_label('m/s',  fontsize=15)
        plt.title('Wind Speed'+' '+'('+dates[k]+')')
        plt.savefig("{}-stream-d{}.png".format(dates[k],i), dpi=600)
        plt.clf()

<Figure size 640x480 with 0 Axes>